In [129]:
import tensorflow as tf
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import json

In [130]:
#讀檔

num = 0
train_data = []
test_data = []
train_size = 10000
test_size = 100
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

input_file = open('translation2019zh_train.json', encoding = 'utf8')
for line in input_file.readlines():
    num = num + 1
    if num > train_size + test_size:
        break
    elif num > train_size:
        test_data.append(json.loads(line))
    else:
        train_data.append(json.loads(line))
input_file.close()

'''
input_file = open('translation2019zh_valid.json', encoding = 'utf8')
for line in input_file.readlines():
    num = num + 1
    #data.append(json.loads(line))
input_file.close()'''

"\ninput_file = open('translation2019zh_valid.json', encoding = 'utf8')\nfor line in input_file.readlines():\n    num = num + 1\n    #data.append(json.loads(line))\ninput_file.close()"

In [131]:
print(len(train_data))
print(len(test_data))

500
5


In [132]:
'''#簡轉繁
from opencc import OpenCC

cc = OpenCC('s2t')

for data in train_data:
    data['chinese'] = cc.convert(data['chinese'])'''

In [133]:
# 讀取資料檔，並將所有單字整理為字典，分別為英文及中文字典，注意，英文為字母的集合，非單字(Word)
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for item in train_data:
    input_text = item['english']
    target_text = '\t' + item['chinese'] + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [134]:
print(target_texts[:10])

['\t爲了更好的銳度，但是附帶的會多一些顆粒度，可以使用這個顯影劑的1：1稀釋液。\n', '\t他還把宣揚自己思想的所謂《綠皮書》稱作“新福音書”。\n', '\t微風推着我去愛撫它的長耳朵\n', '\t它們的先烈們的鮮血是白流了…\n', '\t最後，在1月31日，湖人將前往汽車城底特律挑戰活塞隊，活塞近來在東部排名第二。\n', '\t“真是天造地設的一對——我父親喜歡結交名人，傑姬酷愛金錢，”亞歷山大在婚禮上譏諷道。他和克里斯蒂娜從未同他們的繼母和睦相處過。\n', '\t2006年，沃爾瑪的推薦引擎竟將《人猿星球》與馬丁·路德·金的記錄片配成了一對，爲此沃爾瑪遭到了種族歧視的指控。\n', '\t通過電子探針顯微分析確定貧化渣中主要銅相爲冰銅相。\n', '\t吉姆靠給人擦皮鞋爲生。\n', '\t用甘氨酸模擬膠原，研究間苯二酚-惡唑烷E鞣性基質的形成以及與膠原之間的反應特性。\n']


In [135]:
# 字典排序            
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

In [136]:
# 計算編碼器、解碼器的最大長度
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [137]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 500
Number of unique input tokens: 125
Number of unique output tokens: 2017
Max sequence length for inputs: 232
Max sequence length for outputs: 122


In [138]:
# 以dict儲存字典單字及序號
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [139]:
# 設定編碼器、解碼器input起始值(均為0矩陣)
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

In [140]:
print(input_token_index)

{' ': 0, '!': 1, '"': 2, '$': 3, '%': 4, '&': 5, "'": 6, '(': 7, ')': 8, ',': 9, '-': 10, '.': 11, '/': 12, '0': 13, '1': 14, '2': 15, '3': 16, '4': 17, '5': 18, '6': 19, '7': 20, '8': 21, '9': 22, ':': 23, ';': 24, '<': 25, '>': 26, '?': 27, 'A': 28, 'B': 29, 'C': 30, 'D': 31, 'E': 32, 'F': 33, 'G': 34, 'H': 35, 'I': 36, 'J': 37, 'K': 38, 'L': 39, 'M': 40, 'N': 41, 'O': 42, 'P': 43, 'Q': 44, 'R': 45, 'S': 46, 'T': 47, 'U': 48, 'V': 49, 'W': 50, 'X': 51, 'Y': 52, 'Z': 53, '[': 54, ']': 55, '_': 56, 'a': 57, 'b': 58, 'c': 59, 'd': 60, 'e': 61, 'f': 62, 'g': 63, 'h': 64, 'i': 65, 'j': 66, 'k': 67, 'l': 68, 'm': 69, 'n': 70, 'o': 71, 'p': 72, 'q': 73, 'r': 74, 's': 75, 't': 76, 'u': 77, 'v': 78, 'w': 79, 'x': 80, 'y': 81, 'z': 82, '~': 83, '·': 84, 'ö': 85, 'ü': 86, '–': 87, '—': 88, '‘': 89, '’': 90, '“': 91, '”': 92, '…': 93, '。': 94, '上': 95, '与': 96, '为': 97, '人': 98, '从': 99, '他': 100, '们': 101, '做': 102, '先': 103, '化': 104, '团': 105, '基': 106, '成': 107, '是': 108, '本': 109, '来': 110,

In [141]:
print(target_token_index)

{'\t': 0, '\n': 1, ' ': 2, '"': 3, '%': 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '/': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, '<': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, '[': 48, ']': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'y': 73, 'z': 74, '·': 75, 'ö': 76, 'ü': 77, '—': 78, '‘': 79, '’': 80, '“': 81, '”': 82, '•': 83, '…': 84, '─': 85, '、': 86, '。': 87, '〈': 88, '〉': 89, '《': 90, '》': 91, '「': 92, '」': 93, '一': 94, '丁': 95, '七': 96, '丈': 97, '三': 98, '上': 99, '下': 100, '不': 101, '且': 102, '世': 103, '並': 104, '中': 105, '串': 106, '丹': 107, '主': 108, '久': 109, '之': 11

In [142]:
# 設定 encoder_input、decoder_input對應的順序    
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [143]:
# 建立 encoder LSTM 隱藏層
origin_encoder_inputs = Input(shape=(None, num_encoder_tokens), name='input_0')
origin_encoder = LSTM(latent_dim, return_state=True)
origin_encoder_outputs, origin_state_h, origin_state_c = origin_encoder(origin_encoder_inputs)
origin_encoder_states = [origin_state_h, origin_state_c]

encoder_inputs = Input(shape=(None, num_encoder_tokens), name='input_1')
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
print(encoder_outputs)
print(state_h)
print(state_c)
encoder_outputs.type_spec

KerasTensor(type_spec=TensorSpec(shape=(None, 256), dtype=tf.float32, name=None), name='lstm_7/PartitionedCall:0', description="created by layer 'lstm_7'")
KerasTensor(type_spec=TensorSpec(shape=(None, 256), dtype=tf.float32, name=None), name='lstm_7/PartitionedCall:2', description="created by layer 'lstm_7'")
KerasTensor(type_spec=TensorSpec(shape=(None, 256), dtype=tf.float32, name=None), name='lstm_7/PartitionedCall:3', description="created by layer 'lstm_7'")


TensorSpec(shape=(None, 256), dtype=tf.float32, name=None)

In [144]:
# 捨棄 output，只保留記憶狀態 h 及 c
encoder_states = [state_h, state_c]

In [145]:
# 建立 decoder LSTM 隱藏層
decoder_inputs = Input(shape=(None, num_decoder_tokens), name='input_2')
# We set up our decoder to return full output sequences,
# decoder 記憶狀態不會在訓練過程使用，只會在推論(Inference)使用
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [146]:
# 定義模型，由 encoder_input_data 及 decoder_input_data 轉換為 decoder_target_data 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [147]:
# 訓練
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
7/7 [==============================] - 42s 5s/step - loss: 2.2539 - val_loss: 2.0935
Epoch 2/100
7/7 [==============================] - 29s 4s/step - loss: 2.0624 - val_loss: 2.0852
Epoch 3/100
7/7 [==============================] - 29s 4s/step - loss: 2.0724 - val_loss: 2.0818
Epoch 4/100
7/7 [==============================] - 30s 4s/step - loss: 2.0455 - val_loss: 2.0818
Epoch 5/100
7/7 [==============================] - 27s 4s/step - loss: 2.0443 - val_loss: 2.0801
Epoch 6/100
7/7 [==============================] - 28s 4s/step - loss: 2.0294 - val_loss: 2.0808
Epoch 7/100
7/7 [==============================] - 28s 4s/step - loss: 2.0100 - val_loss: 2.0807
Epoch 8/100
7/7 [==============================] - 28s 4s/step - loss: 2.0117 - val_loss: 2.0805
Epoch 9/100
7/7 [==============================] - 26s 4s/step - loss: 1.9890 - val_loss: 2.0811
Epoch 10/100
7/7 [==============================] - 27s 4s/step - loss: 2.0327 - val_loss: 2.0826
Epoch 11/100
7/7 [===========

7/7 [==============================] - 29s 4s/step - loss: 2.0124 - val_loss: 2.1253
Epoch 85/100
7/7 [==============================] - 40s 5s/step - loss: 2.0121 - val_loss: 2.1227
Epoch 86/100
7/7 [==============================] - 38s 6s/step - loss: 1.9885 - val_loss: 2.1268
Epoch 87/100
7/7 [==============================] - 29s 4s/step - loss: 2.0404 - val_loss: 2.1253
Epoch 88/100
7/7 [==============================] - 28s 4s/step - loss: 2.0202 - val_loss: 2.1284
Epoch 89/100
7/7 [==============================] - 27s 4s/step - loss: 1.9831 - val_loss: 2.1239
Epoch 90/100
7/7 [==============================] - 26s 4s/step - loss: 1.9739 - val_loss: 2.1238
Epoch 91/100
7/7 [==============================] - 29s 4s/step - loss: 1.9587 - val_loss: 2.1253
Epoch 92/100
7/7 [==============================] - 28s 4s/step - loss: 1.9408 - val_loss: 2.1263
Epoch 93/100
7/7 [==============================] - 26s 4s/step - loss: 2.0047 - val_loss: 2.1297
Epoch 94/100
7/7 [===============

In [148]:
# 定義編碼器取樣模型
encoder_model = Model(encoder_inputs, encoder_states)

In [149]:
# 定義解碼器的input
decoder_state_input_h = Input(shape=(latent_dim,), name='input_3')
decoder_state_input_c = Input(shape=(latent_dim,), name='input_4')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [150]:
# 定義解碼器 LSTM 模型
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

In [151]:
# 以編碼器的記憶狀態 h 及 c 為解碼器的記憶狀態  
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

print([decoder_inputs] + decoder_states_inputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
'''
decoder_model = Model(inputs=[decoder_inputs].append(decoder_states_inputs), outputs=[decoder_outputs].append(decoder_states))
'''

[<KerasTensor: shape=(None, None, 2017) dtype=float32 (created by layer 'input_2')>, <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'input_3')>, <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'input_4')>]


'\ndecoder_model = Model(inputs=[decoder_inputs].append(decoder_states_inputs), outputs=[decoder_outputs].append(decoder_states))\n'

In [152]:
# 建立反向的 dict，才能透過查詢將數值轉回文字
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [153]:
# 模型預測，並取得翻譯結果(中文)    
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [154]:
test_texts = []
test_characters = set()
for text in test_data:
    test_texts.append(text['english'])
    for char in text['english']:
        if char not in test_characters:
            test_characters.add(char)

print(test_texts)
print(test_characters)
print(input_characters)

['"There\'s no question soccer will grow in this country. Remember, nothing spurs us on more than losing, " says Virginia Tech historian Lazenby, alluding to the USA\'s early exit from this World Cup.', 'In the latter period, they were armed with Winchester rifles, clubs and knives.', '"One moment seems like three years, such as, " is a beautiful miss distance training. "A wall between presenves love", is made from pure friendship.', 'There was no immediate response from China, but it has previously said it wants peace in the region while also insisting on its sovereignty over disputed areas.', 'Winter 2009-2010 showed a new connectivity between mid-latitude extreme cold and snowy weather events and changes in the wind patterns of the Arctic; the so-called Warm Arctic–Cold Continents pattern.']
{'T', 's', 'd', '0', '9', 'l', 'R', 'V', 'x', '2', 'w', 'y', 'S', '1', ',', 't', 'i', '.', 'I', "'", 'O', 'k', 'a', 'u', 'U', 'C', 'p', 'n', ';', 'f', 'h', 'e', 'g', 'A', 'm', 'z', '"', 'b', '–'

In [155]:
test_characters = sorted(list(test_characters))
num_test = len(test_characters)

In [156]:
test_index = dict([(char, i) for i, char in enumerate(test_characters)])

In [157]:
test_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

In [164]:
# 設定 encoder_input、decoder_input對應的順序    
for i, test_text in enumerate(test_texts):
    for t, char in enumerate(test_text):
        #print(char)
        #print(input_token_index[char])
        test_input_data[i, t, input_token_index[char]] = 1.

In [165]:
print(len(test_input_data))

500


In [166]:
print(test_index)

{' ': 0, '"': 1, "'": 2, ',': 3, '-': 4, '.': 5, '0': 6, '1': 7, '2': 8, '9': 9, ';': 10, 'A': 11, 'C': 12, 'I': 13, 'L': 14, 'O': 15, 'R': 16, 'S': 17, 'T': 18, 'U': 19, 'V': 20, 'W': 21, 'a': 22, 'b': 23, 'c': 24, 'd': 25, 'e': 26, 'f': 27, 'g': 28, 'h': 29, 'i': 30, 'k': 31, 'l': 32, 'm': 33, 'n': 34, 'o': 35, 'p': 36, 'q': 37, 'r': 38, 's': 39, 't': 40, 'u': 41, 'v': 42, 'w': 43, 'x': 44, 'y': 45, 'z': 46, '–': 47}


In [167]:
print(input_token_index)

{' ': 0, '!': 1, '"': 2, '$': 3, '%': 4, '&': 5, "'": 6, '(': 7, ')': 8, ',': 9, '-': 10, '.': 11, '/': 12, '0': 13, '1': 14, '2': 15, '3': 16, '4': 17, '5': 18, '6': 19, '7': 20, '8': 21, '9': 22, ':': 23, ';': 24, '<': 25, '>': 26, '?': 27, 'A': 28, 'B': 29, 'C': 30, 'D': 31, 'E': 32, 'F': 33, 'G': 34, 'H': 35, 'I': 36, 'J': 37, 'K': 38, 'L': 39, 'M': 40, 'N': 41, 'O': 42, 'P': 43, 'Q': 44, 'R': 45, 'S': 46, 'T': 47, 'U': 48, 'V': 49, 'W': 50, 'X': 51, 'Y': 52, 'Z': 53, '[': 54, ']': 55, '_': 56, 'a': 57, 'b': 58, 'c': 59, 'd': 60, 'e': 61, 'f': 62, 'g': 63, 'h': 64, 'i': 65, 'j': 66, 'k': 67, 'l': 68, 'm': 69, 'n': 70, 'o': 71, 'p': 72, 'q': 73, 'r': 74, 's': 75, 't': 76, 'u': 77, 'v': 78, 'w': 79, 'x': 80, 'y': 81, 'z': 82, '~': 83, '·': 84, 'ö': 85, 'ü': 86, '–': 87, '—': 88, '‘': 89, '’': 90, '“': 91, '”': 92, '…': 93, '。': 94, '上': 95, '与': 96, '为': 97, '人': 98, '从': 99, '他': 100, '们': 101, '做': 102, '先': 103, '化': 104, '团': 105, '基': 106, '成': 107, '是': 108, '本': 109, '来': 110,

In [172]:
# 測試100次
for seq_index in range(len(test_texts)):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = test_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(seq_index + 1, '.')
    print('English:', test_texts[seq_index])
    try:
        print('Chinese:', decoded_sentence)
    except:
        # 出現亂碼，以?取代
        print('Chinese:', decoded_sentence.encode('ascii', 'replace'))
        #print("error:", sys.exc_info()[0])


1 .
English: "There's no question soccer will grow in this country. Remember, nothing spurs us on more than losing, " says Virginia Tech historian Lazenby, alluding to the USA's early exit from this World Cup.
Chinese: 在在，，，，的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
2 .
English: In the latter period, they were armed with Winchester rifles, clubs and knives.
Chinese: 在在，，，，的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
3 .
English: "One moment seems like three years, such as, " is a beautiful miss distance training. "A wall between presenves love", is made from pure friendship.
Chinese: 在在，，，，的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的的
4 .
English: There was no immediate response from China, but it has previously said it wants peace in the region while also insisting on

In [171]:
print(test_data)

[{'english': '"There\'s no question soccer will grow in this country. Remember, nothing spurs us on more than losing, " says Virginia Tech historian Lazenby, alluding to the USA\'s early exit from this World Cup.', 'chinese': '“毫无疑问足球将能够在这个国家发展，要知道，没有什么比失败更能刺激我们，”弗吉尼亚工学院历史学家Lazenby说，暗指美国队在本次世界杯中的过早出局。'}, {'english': 'In the latter period, they were armed with Winchester rifles, clubs and knives.', 'chinese': '在后阶段，她们还配备了连发枪以及军棍和军刀。'}, {'english': '"One moment seems like three years, such as, " is a beautiful miss distance training. "A wall between presenves love", is made from pure friendship.', 'chinese': '“一日不见，如隔三秋”，是距离培养了美的思念。“君子之交淡如水”，是距离造就了清纯的友谊。'}, {'english': 'There was no immediate response from China, but it has previously said it wants peace in the region while also insisting on its sovereignty over disputed areas.', 'chinese': '中国尚未对此作出回应，不过中国此前一直说，希望该地区实现和平，同时坚决主张中国对争议地区的主权。'}, {'english': 'Winter 2009-2010 showed a new connectivity between mid-latitude extreme cold and sn